In [9]:
from tokenizer import Tokenizer

tokenizer = Tokenizer()


In [10]:
model = tokenizer.load_model()
model

In [12]:
a = model.encode('I am a president')
a

[15953, 1, 373, 8, 5140]

In [14]:
import torch.nn.functional as F
import torch
onehot = F.one_hot(torch.tensor(a), 16000)
onehot


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [16]:
onehot.shape

torch.Size([5, 16000])

In [7]:
import torch
inputs = torch.rand(64, 20).long()
inputs

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [1]:
from model import EmbeddingLayer

embed = EmbeddingLayer(num_embeddings=20, embedding_dim=512, padding_idx=0, layer_mode='embedding')
embed

EmbeddingLayer(
  (embedding): Embedding(20, 512, padding_idx=0)
  (linear): Embedding(512, 20, padding_idx=0)
)

In [6]:
outputs = embed(inputs)
outputs.shape

torch.Size([64, 20, 512])

In [25]:
import numpy as np
np.sqrt(3)

1.7320508075688772

In [1]:
from model import *
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tokenizer import Tokenizer
from datasets import Dataset
num_embeddings=16000
embedding_dim=512
padding_idx=0
model_dim=512
q_dim=64
k_dim=64
v_dim=64
n_heads=8
in_dim=512
hid_dim=2048
out_dim=512
max_seq_len=20
N=6
mask=True

tokenizer = Tokenizer().load_model()
dataset = Dataset(src='de', trg='en', data_type='train', tokenizer=tokenizer, max_seq_len=20)
train_loader = DataLoader(dataset=dataset, batch_size=64, shuffle=False, collate_fn=dataset.collate_fn)
embed = EmbeddingLayer(num_embeddings=16000, embedding_dim=512, padding_idx=0, layer_mode='embedding')
encoder = Encoder(num_embeddings, embedding_dim, padding_idx, model_dim, q_dim, k_dim, v_dim, n_heads, in_dim, hid_dim, out_dim, max_seq_len, N)
pe = PositionalEncodingLayer()
mha = MultiHeadAttentionLayer(model_dim, q_dim, k_dim, v_dim, n_heads)
decoder = Decoder(num_embeddings, embedding_dim, padding_idx, model_dim, q_dim, k_dim, v_dim, n_heads, in_dim, hid_dim, out_dim, max_seq_len, N, mask)
decoderlayer = DecoderLayer(model_dim, q_dim, k_dim, v_dim, n_heads, in_dim, hid_dim, out_dim, mask)
linear = EmbeddingLayer(num_embeddings=16000, embedding_dim=512, padding_idx=0, layer_mode='linear')
transformer = Transformer(num_embeddings=16000, embedding_dim=512, padding_idx=0, 
                 model_dim=512, q_dim=64, k_dim=64, v_dim=64, n_heads=8,
                 in_dim=512, hid_dim=2048, out_dim=512, max_seq_len=20, N=6)

In [5]:
for epoch in range(1):
    for i, data in enumerate(train_loader):
        inputs, outputs = data
        print("input shape: ", inputs.shape)
        print(outputs.shape)
        temp = transformer(inputs, outputs)
        temp = temp.view(-1, 16000)
        print("output shape: ", temp.shape)
        print(temp)

        break

input shape:  torch.Size([64, 20])
torch.Size([64, 20])
output shape:  torch.Size([1280, 16000])
tensor([[7.2497e-05, 2.0259e-05, 3.6086e-05,  ..., 1.0539e-04, 2.5765e-05,
         3.9717e-05],
        [8.3365e-05, 2.2292e-05, 6.9347e-05,  ..., 1.1280e-04, 1.3574e-05,
         3.0544e-05],
        [1.1523e-04, 4.9236e-05, 4.4523e-05,  ..., 1.1361e-04, 1.3742e-05,
         3.8350e-05],
        ...,
        [7.8031e-05, 2.9709e-05, 5.9426e-05,  ..., 6.5251e-05, 2.7121e-05,
         7.3002e-05],
        [8.5027e-05, 2.3471e-05, 5.9556e-05,  ..., 5.9697e-05, 2.9138e-05,
         6.7957e-05],
        [6.1569e-05, 2.2690e-05, 5.9345e-05,  ..., 5.5302e-05, 3.0623e-05,
         6.4594e-05]], grad_fn=<ViewBackward>)


In [3]:
len(tokenizer)

16000